### План работы

1. Выбрать два 7-10 минутных видео на английском языке (TED talk/интервью/...) с мужским и женским голосом
2. Выполнить предобработку данных: выделить аудио, сегментировать, привести к нужному  для ASR модели формату, при необходимости применить деноизинг
3. Запустить любые open-source ASR и MT модели. Объяснить выбор моделей.
4. Разобраться в архитектуре модели XTTS https://arxiv.org/pdf/2406.04904
5. Zero-shot voice cloning:
   - скопировать репозиторий https://github.com/coqui-ai/TTS
   - сгенерировать аудио для соответствующих текстов на русском языке аудио моделью XTTSv2. В качестве промпта подавать исходные записи на английском для копирования голоса
   - наложить аудио на видео
6. Few-shot voice cloning:  
   - выбрать голос персонажа/актера, на котором zero-shot генерация дает не совсем точное копирование/не передает специфические характеристики, и собрать аудиозаписи для него
   - выполнить файнтюнинг модели на этих данных (проверить на разном количестве данных, подобрать гиперпараметры при необходимости)
   - сгенерировать те же предложения до и после файнтюнинга
   - посчитать объективную метрику похожести голоса (speaker smilarity): косинусное расстояние между векторами модели исходной (gt) и сгенерированной записи для zero- и few-shot вариантов

Результат представить:
  - в виде ноутбука с поэтапным описанием каждого шага/ скриптов
  - описания XTTS модели, основных компонент архитектуры и их задач. описание того, какие части обновляются во время few-shot клонирования
  - описания экспериментов по файнтюнингу, текущих проблем пайплайна
  - исходные и полученные видео

### Настройка окружения

In [1]:
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!git clone https://github.com/coqui-ai/TTS

Cloning into 'TTS'...
remote: Enumerating objects: 32844, done.
remote: Total 32844 (delta 0), reused 0 (delta 0), pack-reused 32844 (from 1)
Receiving objects: 100% (32844/32844), 166.18 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (23832/23832), done.


In [3]:
!pip install -e TTS/. -q

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 59.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 233.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 54.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 17.5

In [1]:
!pip install moviepy -q
!pip install transformers==4.49.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 66.0 MB/s eta 0:00:00


### Предобработка

In [2]:
import os
import librosa
import torch
import datetime
import json
import tempfile
import io
import re
import sys
from contextlib import contextmanager
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip
from pathlib import Path
from tqdm.notebook import tqdm
from pydub import AudioSegment
from pydub.effects import speedup, split_on_silence
from transformers import pipeline
from datasets import load_dataset
from TTS.api import TTS


device = "cuda:0" if torch.cuda.is_available() else "cpu"

  if event.key is 'enter':



In [3]:
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

In [4]:
def convert_video_to_audio_moviepy(video_file: str, output_ext: str = "wav") -> None:
    filename, _ = os.path.splitext(video_file)
    clip = VideoFileClip(str(video_file))
    with suppress_stdout():
        clip.audio.write_audiofile(f"{filename}.{output_ext}")

In [5]:
for video_file in tqdm(list(Path("data").glob("*.mp4"))):
    convert_video_to_audio_moviepy(video_file)

  0%|          | 0/2 [00:00<?, ?it/s]

### ASR

Выбрал Wisper по следующим соображениям:

*   Кажется, лучшая модель для ASR
*   Умеет сама разбивать длинные куски аудио
*   Предсказывает таймстемпы начала и конца каждого кусочка

Перед тем как делать транскрибирование аудио выделил интервалы без тишины с помощью `librosa`.

Экспериментально выявил, что нужно выделять куски аудио длиной 15 сек.

Помимо текста предсказываю таймстемпы с поправкой на начало каждого полученного интервала.



In [6]:
asr_model = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [72]:
def speech_to_text(model, audio_file, chunk_length_s=15):
    array, sampling_rate = librosa.load(audio_file)
    intervals = librosa.effects.split(array)
    result = []
    for interval in intervals:
        result += [
            {
                **data,
                "timestamp": [
                    ts + librosa.samples_to_time(interval[0])
                    for ts in data["timestamp"]
                ]
            }
            for data in model(
                {
                    "array": array[interval[0]:interval[1]],
                    "sampling_rate": sampling_rate
                },
                generate_kwargs={"task": "transcribe"},
                chunk_length_s=chunk_length_s,
                batch_size=8,
                return_timestamps=True,
            )['chunks']
        ]
    return {"chunks": result}

In [73]:
result = {}
for audio_file in tqdm(list(Path("data").glob("*.wav"))):
    result[audio_file.stem] = speech_to_text(asr_model, audio_file)

  0%|          | 0/2 [00:00<?, ?it/s]

In [76]:
with open("stt_data.json", "w") as fout:
    json.dump(result, fout, sort_keys=True, indent=4)

### MT

Перепробовал кучу моделей в итоге остановился на `utrobinmv/t5_translate_en_ru_zh_large_1024_v2`

Плохо показала себя `Helsinki-NLP/opus-mt-en-ru`

Не заработала `facebook/nllb-200-distilled-600M`

Слишком большими оказались:


*   `jbochi/madlad400-7b-mt`
*   `ModelSpace/GemmaX2-28-2B-v0.1`
*   `facebook/wmt21-dense-24-wide-en-x`

Показались нормальными, но хуже чем выбранная:

*   `facebook/wmt19-en-ru`
*   `google/madlad400-3b-mt` (галлюционировала чем-то таким: "10. Осуществление Конвенции о ликвидации всех форм дискриминации в отношении женщин")

`gpt2` не подошла для перевода на русский язык

В ходе перевода делю слишком большие куски текста по 400 символов максимум, иначе модель игнорирует больший input.






In [10]:
mt_model = pipeline(
    "translation",
    model="utrobinmv/t5_translate_en_ru_zh_large_1024_v2",
    device=device
)

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Device set to use cuda:0


In [78]:
stt_data = json.load(open("stt_data.json"))

In [17]:
def translate_text(model, en_text, max_sentence_size=400):
    if en_text.endswith("."):
        en_text = en_text[:-1]
    parts = [t.strip() for t in en_text.split(".")]
    chunks = []
    cumm = ""
    for part in parts:
        if len(". ".join([cumm, part])) > max_sentence_size and cumm:
            chunks.append(cumm)
            cumm = part
        else:
            cumm = ". ".join([cumm, part]) if cumm else part
    if cumm:
        chunks.append(cumm)
    with suppress_stdout():
        translation_text = ". ".join([
            output["translation_text"]
            for output
            in model(["translate to ru:" + chunk for chunk in chunks])
        ])
    return translation_text

In [79]:
for _, data in tqdm(stt_data.items()):
    for chunk in tqdm(data['chunks']):
        chunk["translated"] = translate_text(mt_model, chunk["text"])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/73 [00:00<?, ?it/s]

Your input_length: 23 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 22 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 41 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 28 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 22 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 24 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 23 is bigger than 0.9 * max_length: 20. You might conside

  0%|          | 0/55 [00:00<?, ?it/s]

Your input_length: 61 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 29 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 19 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 33 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 19 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 24 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 26 is bigger than 0.9 * max_length: 20. You might conside

In [80]:
with open("mt_data.json", "w", encoding="utf-8") as fout:
    json.dump(stt_data, fout, sort_keys=True, indent=4, ensure_ascii=False)

### Zero-shot voice clonning

Теперь каждый переведенный кусочек переводим в голос, причем в качестве промпта используем часть аудио, примерно соответствующую этому тексту. В теории это позволит подстроиться под темп и манеру речи говорящего.

Затем объединяем все кусочки аудио в один файл, при этом если нужно ускоряем аудио, но не больше чем в 1.5 раза, иначе получается каша.

В финале накладываем получившееся аудио на исходное видео.

In [81]:
print(TTS().list_models().list_models())

['tts_models/multilingual/multi-dataset/xtts_v2', 'tts_models/multilingual/multi-dataset/xtts_v1.1', 'tts_models/multilingual/multi-dataset/your_tts', 'tts_models/multilingual/multi-dataset/bark', 'tts_models/bg/cv/vits', 'tts_models/cs/cv/vits', 'tts_models/da/cv/vits', 'tts_models/et/cv/vits', 'tts_models/ga/cv/vits', 'tts_models/en/ek1/tacotron2', 'tts_models/en/ljspeech/tacotron2-DDC', 'tts_models/en/ljspeech/tacotron2-DDC_ph', 'tts_models/en/ljspeech/glow-tts', 'tts_models/en/ljspeech/speedy-speech', 'tts_models/en/ljspeech/tacotron2-DCA', 'tts_models/en/ljspeech/vits', 'tts_models/en/ljspeech/vits--neon', 'tts_models/en/ljspeech/fast_pitch', 'tts_models/en/ljspeech/overflow', 'tts_models/en/ljspeech/neural_hmm', 'tts_models/en/vctk/vits', 'tts_models/en/vctk/fast_pitch', 'tts_models/en/sam/tacotron-DDC', 'tts_models/en/blizzard2013/capacitron-t2-c50', 'tts_models/en/blizzard2013/capacitron-t2-c150_v2', 'tts_models/en/multi-dataset/tortoise-v2', 'tts_models/en/jenny/jenny', 'tts_m

In [25]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.87G/1.87G [00:25<00:00, 90.6MiB/s]
100%|██████████| 1.87G/1.87G [00:26<00:00, 71.6MiB/s]
4.37kiB [00:00, 17.9kiB/s]

361kiB [00:00, 1.29MiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 113iB/s]
 63%|██████▎   | 4.88M/7.75M [00:00<00:00, 48.8MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


  self.speakers = torch.load(speaker_file_path)

100%|██████████| 7.75M/7.75M [00:15<00:00, 48.8MiB/s]WARNING:py.warnings:/content/TTS/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to t

In [82]:
mt_data = json.load(open("mt_data.json"))

In [126]:
def get_audio_chunks(model, audio_filename, data):
    audio_filename_dir = Path(audio_filename).with_suffix("")
    audio_filename_dir.mkdir(parents=True, exist_ok=True)
    audio = AudioSegment.from_wav(audio_filename)
    for i, chunk in enumerate(tqdm(data)):
        tmp_en_audio = (
            audio[chunk["timestamp"][0] * 1000:chunk["timestamp"][1] * 1000]
            .export(f"{tempfile.gettempdir()}/audio.wav", format="wav")
        )
        audio_file_name = str(audio_filename_dir / (audio_filename_dir.stem + f"_{i}.wav"))
        with suppress_stdout():
            model.tts_to_file(
                text=re.sub(r"[«»]", "", chunk["translated"]),
                speaker_wav=f"{tempfile.gettempdir()}/audio.wav",
                language="ru",
                file_path=audio_file_name
            )
        chunk["audio"] = audio_file_name

In [129]:
for name, data in tqdm(mt_data.items()):
    get_audio_chunks(tts, f"data/{name}.wav", data["chunks"])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

In [85]:
with open("zero_shot_data.json", "w") as fout:
    json.dump(mt_data, fout, sort_keys=True, indent=4, ensure_ascii=False)

In [86]:
zero_shot_data = json.load(open("zero_shot_data.json"))

In [117]:
def combine_audio(name, data):
    combined_audio = AudioSegment.silent(duration=0)
    for chunk in tqdm(data["chunks"]):
        start, end = chunk["timestamp"]
        if start * 1000 - len(combined_audio) > 0:
            combined_audio += AudioSegment.silent(duration=start * 1000 - len(combined_audio))
        audio = AudioSegment.from_wav(chunk["audio"])
        rate = len(audio) / (end - start) / 1000
        combined_audio += speedup(audio, min(rate, 1.5)) if rate > 1 else audio
    combined_audio.export(f"data/{name}_translated.wav", format="wav")
    data["audio"] = f"data/{name}_translated.wav"

In [114]:
for name, data in tqdm(zero_shot_data.items()):
    combine_audio(name, data)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

In [118]:
def audio_to_video(video, audio):
    dubbed_video_path = str(Path(video).with_suffix("")) + "_dubbed.mp4"
    video_clip = VideoFileClip(video)
    audio_clip = AudioFileClip(audio)
    audio_clip = audio_clip.volumex(2)
    final_audio = CompositeAudioClip([video_clip.audio, audio_clip])
    final_clip = video_clip.set_audio(final_audio)
    with suppress_stdout():
        final_clip.write_videofile(
            dubbed_video_path,
            codec='libx264',
            audio_codec='aac',
        )
    return dubbed_video_path

In [120]:
for name, data in tqdm(zero_shot_data.items()):
    data["zero_shot_video"] = audio_to_video(f"data/{name}.mp4", data["audio"])

  0%|          | 0/2 [00:00<?, ?it/s]

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



In [121]:
with open("zero_shot_data.json", "w") as fout:
    json.dump(mt_data, fout, sort_keys=True, indent=4, ensure_ascii=False)

## Все вместе

In [123]:
Path("data/asdasd.as").with_suffix(".wav")

PosixPath('data/asdasd.wav')

In [132]:
def run(video_file, asr_model, mt_model, tts_model):
    convert_video_to_audio_moviepy(video_file)
    audio_file = Path(video_file).with_suffix(".wav")
    chunks = speech_to_text(asr_model, audio_file)["chunks"]
    for chunk in tqdm(chunks):
        chunk["translated"] = translate_text(mt_model, chunk["text"])
    get_audio_chunks(tts_model, str(audio_file), chunks)
    data = {"chunks": chunks}
    combine_audio(audio_file.stem, data)
    data["zero_shot_video"] = audio_to_video(video_file, data["audio"])
    print(data["zero_shot_video"])
    return data

In [130]:
cameron_out = run("data/Cameron.mp4", asr_model, mt_model, tts)

  warnings.warn(



  0%|          | 0/55 [00:00<?, ?it/s]

Your input_length: 61 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 29 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 19 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 33 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 19 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 24 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 26 is bigger than 0.9 * max_length: 20. You might conside

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

  warnings.warn("Warning: in file %s, "%(self.filename)+



data/Cameron_dubbed.mp4


In [131]:
bruno_out = run("data/Bruno.mp4", asr_model, mt_model, tts)

  warnings.warn(



  0%|          | 0/73 [00:00<?, ?it/s]

Your input_length: 23 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 22 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 41 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 28 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 22 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 24 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 23 is bigger than 0.9 * max_length: 20. You might conside

  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/73 [00:00<?, ?it/s]

  warnings.warn("Warning: in file %s, "%(self.filename)+



data/Bruno_dubbed.mp4


## Результаты и выводы

1. Самым сложным оказалось наложить дублированные кусочки аудио на видео, перепробовал много подходов: ускорение/замедление всего аудио, нарезание исходного аудио на маленькие кусочки и работа с ними, поиск тишины в исходном аудио и в получившемся, в итоге остановился на:


*   Первоначальный поиск тишины с помощью `librosa`
*   Нарезка аудио на сегменты и предсказание таймстемпов возложил на `Wisper`
*   При комбинировании аудио делал только ускорение, где необходимо, и скорость ограничил в `1.5` раза максимум

Получилось все равно не идеально.

2. В дубляже очень много зависит от качества перевода, к сожалению не удалось найти модель на Hugging Face, которая бы идеально подошла. Плюс проблемой было то, что переводились отдельные кусочки транскрибированного текста, что привело к потере контекста на границах.

3. Пришлось прибегнуть к некоторым эвристикам, которые возможно не подойдут к другим видео.

4. К сожалению, не успел сделать часть с Few-shot voice cloning.

